In [2]:
import re

class Morph():
    def __init__(self, doc_list):
        self.surface = doc_list[0]
        self.base = doc_list[7]
        self.pos = doc_list[1]
        self.pos1= doc_list[2]

    def __str__(self):
        return "{" + f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}" + "}"

class Chunk():
    def __init__(self,morph_list,dst,srcs):
        self.morph_list = morph_list
        self.dst = int(dst)
        self.srcs = srcs

    def __str__(self):
        morphs_str = ', '.join(str(morph) for morph in self.morph_list) # Morphオブジェクトのリストを文字列に変換
        return f"Morphs: [{morphs_str}], Dst: {self.dst}, Srcs: {self.srcs}"

# 文節番号と係り先番号の取得
def find_feature_number(text):
    pattern = r"^\* \d+ ([^\d]*\d+)D"
    match = re.search(pattern, text)
    if match:
        dst_number = match.group(1)  # 文節番号  # 係先番号
        return dst_number
    else:
        return None

In [3]:
with open(file="ai.ja.txt.parsed", mode="r", encoding="utf-8")as f:
    docs = f.readlines()

Chunk_list = []
chunk_flag = False
# srcの追加処理は後で行う
for doc in docs[5:]:
    if doc == "EOS\n":
        chunk = Chunk(morph_list,dst_num,[])
        Chunk_list.append(chunk)
        break
    # *から始まる行とEOSの行は飛ばす
    if doc[0]=="*":
        if chunk_flag:
            chunk = Chunk(morph_list,dst_num,[])
            Chunk_list.append(chunk)
        dst_num = find_feature_number(doc)
        morph_list = []
        chunk_flag = True
    else:
        doc = doc.replace("\t",",").replace("\n", "")
        doc_list = doc.split(",")
        doc_morph = Morph(doc_list)
        morph_list.append(doc_morph)


# Srcsを埋めていく
for i in range(len(Chunk_list)):
    Chunk_list[Chunk_list[i].dst].srcs.append(i)

In [16]:
# morphの最左の動詞を抽出，なければNoneを返す
def get_verb_in_sentence(chunk):
    for morph in chunk.morph_list:
        if morph.pos == "動詞":
            return morph.base
    return None

# ppじは助詞(postpositional particle)のこと
# 助詞のリストを返す
def get_pp_in_sentence(chunk):
    for morph in chunk.morph_list:
        if morph.pos == "助詞":
            return morph.base
    return None

def get_sentence(chunk):
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    sentence = ""
    for morph in chunk.morph_list:
        if morph.surface not in remove_list:
            sentence += morph.surface
    return sentence

def get_sahen_josi_in_sentence(chunk):
    for i in range(len(chunk.morph_list)-1):
        if chunk.morph_list[i].pos1 == "サ変接続" and chunk.morph_list[i+1].pos == "助詞":
            return chunk.morph_list[i].surface + chunk.morph_list[i+1].surface
    return None


for chunk in Chunk_list:
    has_verb = get_verb_in_sentence(chunk)
    if has_verb == None:    #動詞がなかったら飛ばす
        continue
    #動詞があれば，その文節に係っている(srcs)助詞を抽出する
    srcs_pp_list = []
    srcs_sentence_list = []
    sahen_josi_flag = False #サ変接続+助詞が存在するかのフラッグ
    # srcsの中の文節でサ変接続+助詞があるか確認，あれば抽出，なければNone
    for src in chunk.srcs:
        has_sahen_josi = get_sahen_josi_in_sentence(Chunk_list[src])
        if has_sahen_josi != None:
            #サ変+助詞を変数に格納
            sahen_josi = has_sahen_josi
            sahen_josi_flag = True

    if sahen_josi_flag == False:
        continue
    for src in chunk.srcs:
        has_sahen_josi = get_sahen_josi_in_sentence(Chunk_list[src])
        if has_sahen_josi == None:
            has_pp = get_pp_in_sentence(Chunk_list[src])
            if has_pp != None:
                srcs_pp_list.append(has_pp)
                srcs_sentence_list.append(get_sentence(Chunk_list[src]))

    if srcs_pp_list != []:
        print(f"{sahen_josi}{has_verb}\t",end = "")
        pairs = zip(srcs_pp_list,srcs_sentence_list)
        sorted_pairs = sorted(pairs)
        sorted_srcs_pp_list, sorted_srcs_sentence_list = zip(*sorted_pairs)
        for pp in sorted_srcs_pp_list:
            print(pp, end="")
        print("\t", end="")
        for sentence in sorted_srcs_sentence_list:
            print(sentence, end = "")
        print()


行動を代わる	に	人間に
